In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
pd.set_option('display.max_columns', None)
import gc
import warnings
warnings.filterwarnings('ignore')


In [2]:
trad = pd.read_parquet('cibil_base_nov_aug.parquet.gzip')
# enq = pd.read_parquet('enquiriesdump.parquet.gzip')

In [3]:
trad['employmentTypeId'] = pd.to_numeric(trad['employmentTypeId'], errors = 'coerce')
trad.loc[trad['employmentTypeId']==1, 'Emp_Type'] = 'Salaried'
trad.loc[trad['employmentTypeId'].isin([2,3]), 'Emp_Type'] = 'Self Employed'


In [4]:
pl = ['Personal Loan', 'Microfinance Personal Loan', 'P2P Personal Loan']
bl = ['Business Loan General', 'Business Loan - General','(BLPS-AGR) Business Loan â€“ Priority Sector â€“ Agriculture',
    'Business Loan - Unsecured','(BLPS-SB) Business Loan â€“ Priority Sector â€“ Small Business',
    'Business Loan Unsecured','(BLPS-OTH) Business Loan â€“ Priority Sector â€“ Others','Business Loan - Secured',
    'Microfinance Business Loan','Loan to Professional','(BNFCF-PS-SB) Business Non-Funded Credit Facility â€“ Priority Sector â€“ Small Business',
    '(BNFCF-PS-AGR)  Business Non-Funded Credit Facility â€“ Priority Sector â€“ Agriculture',
    '(BNFCF-GEN) Business Non-Funded Credit Facility â€“ General','(BNFCF-PS-OTH)  Business Non-Funded Credit Facility â€“ Priority Sector-Others',
    "(BLABD) Current Business Loan Against Bank Deposits",'Business Loan - Priority Sector - Small Business',
    'Business Loan - Priority Sector - Agriculture','Business Loan - Priority Sector - Others','Microfinance - Business Loan',
    'Business Non-Funded Credit Facility - General','Business Non-Funded Credit Facility-Priority Sector-Agriculture',
    'Business Loan Against Bank Deposits','Business Non-Funded Credit Facility-Priority Sector- Small Business',
    'Business Non-Funded Credit Facility-Priority Sector-Others']
cc = ['Corporate Credit Card', 'Credit Card', 'Secured Credit Card']
# al = ['Auto Loan (Personal)','Commercial Vehicle Loan','Two-Wheeler Loan','Used Car Loan','Used Tractor Loan']
al = ['Auto Loan (Personal)','Commercial Vehicle Loan']
hl = ['Housing Loan', 'Microfinance Housing Loan']
cl = ['Consumer Loan']
gl = ['Gold Loan']
# el = ['Education Loan']
# others = ['(LABD) Current Loan Against Bank Deposits','(NFCF) Non-Funded Credit Facility',
#           'Construction Equipment Loan','Loan Against Shares / Securities','Loan on Credit Card',
#           'Microfinance Others','Mudra Loans – Shishu / Kishor / Tarun','Other',
#           'Pradhan Mantri Awas Yojana - CLSS']
unsecured = ['Personal Loan','Microfinance Personal Loan','P2P Personal Loan','Business Loan General',
             'Business Loan - General','(BLPS-AGR) Business Loan â€“ Priority Sector â€“ Agriculture',
             'Overdraft','Business Loan - Unsecured','(BLPS-SB) Business Loan â€“ Priority Sector â€“ Small Business',
             'Business Loan Unsecured''(BLPS-OTH) Business Loan â€“ Priority Sector â€“ Others','Microfinance Business Loan',
             'Loan to Professional','(BNFCF-PS-SB) Business Non-Funded Credit Facility â€“ Priority Sector â€“ Small Business',
             '(BNFCF-PS-AGR)  Business Non-Funded Credit Facility â€“ Priority Sector â€“ Agriculture',
             '(BNFCF-GEN) Business Non-Funded Credit Facility â€“ General',
             '(BNFCF-PS-OTH)  Business Non-Funded Credit Facility â€“ Priority Sector-Others',
             'Overdraft','Business Loan - Priority Sector - Small Business','Business Loan - Priority Sector - Agriculture',
             'Business Loan - Priority Sector - Others','Microfinance - Business Loan''Business Non-Funded Credit Facility - General',
             'Business Non-Funded Credit Facility-Priority Sector-Agriculture','Business Non-Funded Credit Facility-Priority Sector- Small Business',
             'Business Non-Funded Credit Facility-Priority Sector-Others','Consumer Loan','Education Loan','Corporate Credit Card',
             'Credit Card','(NFCF) Non-Funded Credit Facility','Construction Equipment Loan','Loan on Credit Card','Microfinance Others',
             'Mudra Loans – Shishu / Kishor / Tarun','Other','Pradhan Mantri Awas Yojana - CLSS']


In [5]:
def live_trd(i):
    trad[f'total_{i}'] = 0
    trad[f'live_{i}'] = 0
    trad.loc[trad['accountTypeName'].isin(accType_dict[i]), f'total_{i}'] = 1
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & ((trad['dateClosed'].isna()) | (trad['dateClosed'] == 'NaT')),f'live_{i}'] = 1
def vintage(i):
    trad[f'{i}_vintage'] = 0
    trad[f'live_{i}_vintage'] = 0
    trad[f'{i}_last_active'] = np.nan
    trad[f'{i}_last_closed'] = np.nan
    trad.loc[trad['accountTypeName'].isin(accType_dict[i]), f'{i}_vintage'] = trad["Vintage"]
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & ((trad['dateClosed'].isna()) | (trad['dateClosed'] == 'NaT')), f'live_{i}_vintage'] = trad["Vintage"]
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i]) & (trad['accountStatusId'] == 17 )), f'{i}_last_active'] = (trad['reportDate'] - trad['openDate']).dt.days
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i]) & (trad['accountStatusId'] == 18 )), f'{i}_last_closed'] = (trad['reportDate'] - trad['dateClosed']).dt.days
def new_acc(i):
    trad[f"new_{i}_3m"] = 0
    trad[f"new_{i}_6m"] = 0
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & (trad['Vintage'] <= 3), f"new_{i}_3m"] = 1
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & (trad['Vintage'] <= 6), f"new_{i}_6m"] = 1
def amount(i):
    trad[f'sum_{i}_limit'] = 0
    trad[f'active_max_{i}_limit'] = 0
    trad[f'max_{i}_limit'] = 0
    trad[f'active_sum_{i}_limit'] = 0
    trad[f'sum_{i}_limit_last_6m'] = 0
    trad[f'max_{i}_limit_last_6m'] = 0
    trad.loc[trad['accountTypeName'].isin(accType_dict[i]),f'sum_{i}_limit'] = trad['creditLimit']
    trad.loc[trad['accountTypeName'].isin(accType_dict[i]),f'max_{i}_limit'] = trad['creditLimit']
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & ((trad['dateClosed'].isna()) | (trad['dateClosed'] == 'NaT')),f'active_sum_{i}_limit'] = trad['creditLimit']
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & ((trad['dateClosed'].isna()) | (trad['dateClosed'] == 'NaT')),f'active_max_{i}_limit'] = trad['creditLimit']
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & (trad['Vintage'] <= 6) ,f'sum_{i}_limit_last_6m'] = trad['creditLimit']
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & (trad['Vintage'] <= 6) ,f'max_{i}_limit_last_6m'] = trad['creditLimit']
def remaining(i):
    trad[f'{i}_remaining_balance'] = 0
    trad[f'mean_{i}_remaining_balance_per'] = 0
    trad[f'max_{i}_remaining_balance_per'] = 0
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & ((trad['dateClosed'].isna()) | (trad['dateClosed'] == 'NaT')),f"{i}_remaining_balance"] = trad['currentBalance']
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & ((trad['dateClosed'].isna()) | (trad['dateClosed'] == 'NaT')),f"mean_{i}_remaining_balance_per"] = trad['currentBalance'] / trad['highCredit']
    trad.loc[(trad['accountTypeName'].isin(accType_dict[i])) & ((trad['dateClosed'].isna()) | (trad['dateClosed'] == 'NaT')),f"max_{i}_remaining_balance_per"] = trad['currentBalance'] / trad['highCredit']
### dpd based
### code to do padding
### pass original dataframe in this
def dpd_padding(dataframe) :
    dataframe['reportDate'] = pd.to_datetime(dataframe['reportDate'] ,  format = "%Y-%m-%d")
    dataframe['dateReported'] = pd.to_datetime(dataframe['dateReported'] , format = "%Y-%m-%d")
    dataframe['accountStatusId'] = pd.to_numeric(dataframe['accountStatusId'], errors='coerce')
    bureau = dataframe
    bureau['month_diff'] = (bureau['reportDate']-bureau['dateReported'])/np.timedelta64(1, 'M')
    bureau['month_diff'] = pd.to_numeric(bureau['month_diff'], errors= 'coerce',downcast="integer").fillna(0).astype(int)
    bureau['aded_payment_hist'] = bureau['month_diff'].apply(lambda x: ["0"]*x)
    del bureau['month_diff']
    gc.collect()
    bureau['paymentHistoryProfile'] = bureau['paymentHistoryProfile'].apply(lambda x: x.split(','))
    bureau['final_paymentHistoryProfile'] = bureau['aded_payment_hist'] + bureau['paymentHistoryProfile']
    del bureau['aded_payment_hist']
    gc.collect()
    bureau['final_paymentHistoryProfile'] = bureau['final_paymentHistoryProfile'].apply(lambda x: [0 if i == '' else i for i in x]).apply(lambda x: [int(i) for i in x])
    return bureau
def dpd_feat(months):
    trad[f"{months}_months"] = trad['final_paymentHistoryProfile'].apply(lambda x: x[0 : months])
    trad[f"{months}_months_max_dpd"] = trad[f"{months}_months"].apply(lambda x : max(x))
    trad[f"{months}_months_dpd_instances"] = trad[f"{months}_months"].apply(lambda x : [0 if i == 0 else 1 for i in x]).apply(lambda x : sum(x))
    del trad[f"{months}_months"]
def weight_dpd(x):
    i = len(x)
    z = 0
    for count, j in enumerate(x):
        z = z + (j * (i - count)) / i
    z = round(z,2)
    return z
def dpd_type(i):
    trad[f"{i}_weighted_dpd_6m"] = 0
    trad[f"{i}_weighted_dpd_12m"] = 0
    trad.loc[trad['accountTypeName'].isin(accType_dict[i]),f"{i}_weighted_dpd_6m"] = trad['weighted_dpd_6m']
    trad.loc[trad['accountTypeName'].isin(accType_dict[i]),f"{i}_weighted_dpd_12m"] = trad['weighted_dpd_12m']


In [6]:
trad1 = pd.DataFrame()

trad['customer_number'] = trad['mobilePhone']
del trad['mobilePhone']
gc.collect()

trad['accountTypeId'] = trad['accountTypeId'].fillna(1).astype(float)

for i in list(trad.dtypes.reset_index().loc[trad.dtypes.reset_index()[0] == 'object']['index']):
    trad.loc[trad[i] == 'NaT', i] = np.nan
    trad.loc[trad[i] == 'nan', i] = np.nan


print("Feat Started")

## max dpd ever
trad['paymentHistoryProfile_for_dpd_feat'] = trad['paymentHistoryProfile'].apply(lambda x: x.replace('XXX','0'))
trad['paymentHistoryProfile_for_dpd_feat'] = trad['paymentHistoryProfile_for_dpd_feat'].apply(lambda x: x.replace('STD','0'))
trad['paymentHistoryProfile_for_dpd_feat'] = trad['paymentHistoryProfile_for_dpd_feat'].apply(lambda x: x.replace('SMA','60'))
                                                                                                                                               

trad['paymentHistoryProfile_for_dpd_feat'] = trad['paymentHistoryProfile_for_dpd_feat'].apply(lambda x: x.split(','))
trad['paymentHistoryProfile_for_dpd_feat'] = trad['paymentHistoryProfile_for_dpd_feat'].apply(lambda x: [0 if i == '' else i for i in x]).apply(lambda x: [int(i) for i in x])

trad['max_dpd_ever'] = trad['paymentHistoryProfile_for_dpd_feat'].apply(lambda x: max(x))
trad['max_dpd_ever'] = pd.to_numeric(trad['max_dpd_ever'], errors = 'coerce')

## count of 30 plus ever
trad['30_plus_ever_count'] = trad['paymentHistoryProfile_for_dpd_feat'].apply(lambda x : [1 if i >= 30 else 0 for i in x]).apply(lambda x : sum(x))

## accountHolderType
trad['guarantor'] = np.where(trad['accountHolderTypeId'].isin(['3','3.0']), 1, np.nan)
trad['joint'] = np.where(trad['accountHolderTypeId'].isin(['4','4.0']), 1, np.nan)

## count of active stpl
trad['highCredit'] = pd.to_numeric(trad['highCredit'], errors = 'coerce')
trad['count_of_active_stpl'] = np.where(((trad['accountTypeName']=='Personal Loan')&(trad['accountStatusName']=='Active')&(trad['highCredit']<=50000)), 1, np.nan)

## days since last payment
# trad['days_since_last_payment'] = (pd.to_datetime(trad['disbursal_date'], errors = 'coerce') - pd.to_datetime(trad['dateofLastPayment'], errors = 'coerce')).dt.days


## Imputed Income
trad['creditLimitAmount'] = pd.to_numeric(trad['creditLimitAmount'], errors='coerce')
trad['highCredit'] = pd.to_numeric(trad['highCredit'] , errors='coerce')
trad['max_limit'] = trad[['creditLimitAmount','highCredit']].max(axis=1)
trad['max_limit']  = trad['max_limit'].fillna(0).astype(int)

# Income Cut
v=[]
for i in trad["accountTypeName"]:
    if i in(["Credit Card","Corporate Credit Card","Fleet Card","Kisan Credit Card","Secured Credit Card"]):
        v.append(1)
    elif i in(["Personal Loan"]):
        v.append(2)
    elif i in(["Auto Loan","Used Car Loan","Two-Wheeler Loan","Auto Loan (Personal)",'Auto Loan (Personal)']):
        v.append(3)
    elif i in("Housing Loan","Pradhan Mantri Awas Yojana - CLSS","Housing Loan"):
        v.append(4)
    else:
        v.append(0)

trad["flag"]=v

s=[]
for i,j in zip(trad["flag"],trad["max_limit"]):
    if i==1:
        temp=j/3
        s.append(temp)
    elif (i==2) & (j<=200000):
        temp=j/5
        s.append(temp)
    elif (i==2) & ((j>200000) & (j<=500000)):
        temp=j/8
        s.append(temp)
    elif (i==2) & (j>500000):
        temp=j/9
        s.append(temp)
    elif (i==3) & (j<100000):
        temp=j/4
        s.append(temp)
    elif (i==3) & ((j>100000) & (j<=200000)):
        temp=j/5
        s.append(temp)
    elif (i==3) & ((j>200000) & (j<=300000)):
        temp=j/7
        s.append(temp)
    elif (i==3) & ((j>300000) & (j<=500000)):
        temp=j/10
        s.append(temp)
    elif (i==3) & (j>500000):
        temp=j/12
        s.append(temp)
    elif (i==4) & (j<=2500000):
        temp=j/3
        s.append(temp)
    elif (i==4) & (j>2500000):
        temp=j/4
        s.append(temp)
    else:
        s.append(0)

trad["Income"] = s
k=[]
for i,j in zip(trad["flag"],trad["Income"]):
    if (i==1) or (i==2) or (i==3):
        temp=j
        k.append(temp)
    else:
        temp=j/12
        k.append(temp)

trad["Income"] = k

rpc = trad.sort_values(['customer_number','flag', 'Income'], ascending=False)
rpc = rpc.loc[rpc['accountStatusName']=='Active'][['customer_number','Income']].drop_duplicates('customer_number')
trad = trad.drop('Income', axis=1).merge(rpc, how='left', on='customer_number')

## Derog
trad['accountStatusId'] = pd.to_numeric(trad['accountStatusId'], errors='coerce')
trad['derog'] = 0
trad.loc[~trad['accountStatusId'].isin([17,18]),'derog'] = 1

print("trad feat")

## Number of tradlines
trad['no_of_trades'] = 1
trad['live_trad'] = 0
trad.loc[trad['accountStatusId'] == 17, 'live_trad'] = 1

d1 = ["pl", "bl", "cc", "cl", "hl", "al"]
d2 = [pl, bl, cc, cl, hl, al]

accType_dict = dict(zip(d1, d2))
del d1, d2

trad['dateClosed'] = pd.to_datetime(trad['dateClosed'])


### Live Tradelines
for i in accType_dict:
    live_trd(i)

## Vintage
trad['reportDate'] = pd.to_datetime(trad['reportDate'])
trad['openDate'] = pd.to_datetime(trad['openDate'])

trad["Vintage"] = (trad["reportDate"].dt.year - trad["openDate"].dt.year) * 12 + (trad["reportDate"].dt.month - trad["openDate"].dt.month)

for i in accType_dict:
    vintage(i)

### New Accounts
import itertools
for i in dict(itertools.islice(accType_dict.items(), 0 ,3)):
    new_acc(i)

print("amount Based")
### Amount Based
trad['creditLimit'] = trad[['creditLimitAmount', 'highCredit']].max(axis = 1)
for i in dict(itertools.islice(accType_dict.items(), 0 ,3)):
    amount(i)

### remaining balance feat
trad['currentBalance'] = trad['currentBalance'].astype(float)

remaining('pl')
remaining('bl')
remaining('al')
remaining('hl')

###  credit Utilization
trad.loc[((trad['accountTypeName'].isin(cc)) & (trad['accountStatusId'] == 17)), "max_credit_utilization"] = trad['highCredit']/trad['creditLimit']
trad.loc[((trad['accountTypeName'].isin(cc)) & (trad['accountStatusId'] == 17)), "mean_credit_utilization"] = trad['highCredit']/trad['creditLimit']

## No of ALHLLAPPLCCBLSBL
list_of_account_types = [1, 2, 3, 5, 10, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 68, 69, 66]
trad.accountTypeId.fillna(0,inplace=True)
trad.accountTypeId = trad.accountTypeId.astype('int')

trad['No_of_ALHLLAPPLCCBLSBL'] = 0
trad.loc[trad['accountTypeId'].isin(list_of_account_types),'No_of_ALHLLAPPLCCBLSBL'] = 1

## RBL_ultrahni
trad['RBL_ultrahni'] = np.where(
    (((trad['accountTypeId'] == 1) &
      (trad['highCredit'] > 1500000)) |
     ((trad['accountTypeId'] == 2) &
      (trad['highCredit'] > 50000000)) |
     ((trad['accountTypeId'] == 3) &
      (trad['highCredit'] > 50000000))), 1, 0)

## TotalOverDueAmount
trad['TotalOverDueAmount'] = trad['amountPastDue']

print("DPD Based")
### dpd Based
trad = dpd_padding(trad)

for i in ([1, 2,3,6,9 ,12]):
    dpd_feat(i)

### weighted DPD
trad["6m_paymentHistoryProfile"] = trad["final_paymentHistoryProfile"].apply(lambda x: x[0 : 6])
trad["6m_paymentHistoryProfile"] = trad["6m_paymentHistoryProfile"].apply(lambda x : x + (6 - len(x))*[0])

trad["12m_paymentHistoryProfile"] = trad["final_paymentHistoryProfile"].apply(lambda x: x[0 : 12])
trad["12m_paymentHistoryProfile"] = trad["12m_paymentHistoryProfile"].apply(lambda x : x + (12 - len(x))*[0])

trad['weighted_dpd_6m'] = trad["6m_paymentHistoryProfile"].apply(lambda x : weight_dpd(x))
trad['weighted_dpd_12m'] = trad["12m_paymentHistoryProfile"].apply(lambda x : weight_dpd(x))

dpd_type("pl")  
dpd_type("bl")
dpd_type("cl")
dpd_type("cc")

print("DBR")

### DBR
trad['currentBalance'] = pd.to_numeric(trad['currentBalance'],errors='coerce')
trad['score'] = pd.to_numeric(trad['score'],errors='coerce')
trad['Age'] = pd.to_numeric(trad['Age'],errors='coerce')

bur_data = trad.copy(deep = True)

bur_data.drop_duplicates(['customer_number','openDate','accountTypeName','creditLimitAmount'],inplace=True)
bur_data.drop_duplicates(['customer_number','openDate','accountTypeName','highCredit'],inplace=True)



from datetime import date

def emi(rate, loan_tenure,loan_amount,outstanding_amount):
    monthly_emi_label=(loan_amount*rate*(1+rate)**loan_tenure/((1+rate)**loan_tenure-1))
    return monthly_emi_label

bur_data.fillna(0,inplace=True)

a=[]
for i,j,k,l in zip(bur_data["accountTypeName"],bur_data["currentBalance"],bur_data["highCredit"],bur_data["Emp_Type"]):

    if i=='Auto Loan (Personal)':
        temp=emi(0.12/12,60,k,0)
        a.append(temp)      
    elif (i=='Consumer Loan') & (l=="Salaried"):
        temp=emi(0.001/12,6,k,0)
        a.append(temp)
    elif i=='Education Loan':
        temp=emi(0.09/12,84,k,0)
        a.append(temp)
    elif i=='Gold Loan':
        temp=emi(0.18/12,33,k,0)
        a.append(temp)
    elif i=='Housing Loan':
        temp=emi(0.09/12,240,k,0)
        a.append(temp)
    elif i=='Microfinance Housing Loan':
        temp=emi(0.14/12,84,k,0)
        a.append(temp)
    elif i=='Microfinance Others':
        temp=emi(0.20/12,60,k,0)
        a.append(temp)
    elif i=='Microfinance Personal Loan':
        temp=emi(0.20/12,60,k,0)
        a.append(temp)
    elif (i=='Personal Loan') & (k>100000):
        temp=emi(0.14/12,60,k,0)
        a.append(temp)
    elif (i=='Personal Loan') & (k<=100000) & (k>30000):
        temp=emi(0.25/12,12,k,0)
        a.append(temp)
    elif (i=='Personal Loan') & (k<=30000):
        temp=emi(0.32/12,3,k,0)
        a.append(temp)    
    elif i=='Property Loan':
        temp=emi(0.11/12,180,k,0)
        a.append(temp)
    elif i=='Secured Credit Card':
        temp=0.05*j
        a.append(temp)
    elif i=='Two-Wheeler Loan':
        temp=emi(0.15/12,48,k,0)
        a.append(temp)
    else:
        a.append(0)
bur_data["EMI"]=a
del a
bur_data['EMI']=bur_data['EMI'].astype(int)

base_1 = bur_data.copy()
# del bureau1
gc.collect()

for i in range(0,36):
    exec('m' + str(i) +' = pd.DataFrame()')

base_1['openDate'] = pd.to_datetime(base_1['openDate'],errors='coerce')
base_1['dateClosed'] = pd.to_datetime(base_1['dateClosed'],errors='coerce')

for i in range(0,36):
    curr=pd.to_datetime((date.today()-pd.DateOffset(months=i)).strftime("%Y-%m-%d"))
    lmt=pd.to_datetime((date.today()-pd.DateOffset(months=(i+1))).strftime("%Y-%m-%d"))
    exec('m' + str(i) + ' = base_1[((base_1.openDate<lmt) & ((base_1.dateClosed>=curr)|(base_1.dateClosed.isna())))][["EMI","customer_number"]]')
    exec('m' + str(i) + '=' + 'm' + str(i) + '.groupby(["customer_number"], as_index=False)["EMI"].sum()')

mfs=[]
for i in range(36):
    exec('m' + str(i)+ '.columns=["customer_number","m'+ str(i) + '"]')
    exec('mfs.append(m' + str(i)+')')
mdfs = [mf.set_index('customer_number') for mf in mfs]
final=mdfs[0].join(mdfs[1:], how="outer")
for i in range(36):
    exec('del m' +str(i))

final.fillna(0,inplace=True)
final2=final.loc[:, final.columns != 'customer_number'].max(axis=1).reset_index()
final2.columns=['customer_number','max_EMI']
final2['m0']=list(final['m0'])
final3=final.idxmax(axis=1).reset_index()
final3.columns=['customer_number','mxmth']
final2=pd.merge(final2,final3,on='customer_number')
final2['mth']=final2['mxmth'].astype(str).str[1:].astype(int)
final2=final2[['customer_number','m0','max_EMI', 'mxmth', 'mth']]

a=[]
for i in base_1["accountTypeName"]:
    if i in(["Credit Card"]):
        a.append(1)
    elif i in("Housing Loan","Pradhan Mantri Awas Yojana - CLSS"):
        a.append(4)
    else:
        a.append(0)

base_1["flag"]=a

a=[]
b=[]
for i,j,k in zip(base_1["flag"],base_1["highCredit"],base_1["creditLimitAmount"]):
    if (i==1)&(k>1000):
        temp=(k/3)
        a.append(temp)
        b.append('limit/3')
    elif (i==1)&(k<1000):
        temp=j/2.5
        a.append(temp)
        b.append('highcredit/2.5')
    else:
        a.append(0)
        b.append('no cc')

base_1["CC_income"]=a
base_1["CC_income_type"]=b
del a,b

a=[]
for i,n in zip(base_1["flag"],base_1['openDate']):
    if (i==1):
        temp= n
        a.append(temp)
    else:
        a.append(0)

base_1["CC_openDate"]=a
hl=base_1[(base_1.flag==4)][['customer_number']].drop_duplicates('customer_number')
hl['flg']=1

final2=pd.merge(final2,hl,on='customer_number',how='left')

final2

gp1=base_1[['customer_number','score','Age']].drop_duplicates('customer_number')

base_1.CC_income = pd.to_numeric(base_1["CC_income"],errors='coerce',downcast='integer')
base_1.CC_openDate = pd.to_datetime(base_1['CC_openDate'], errors='coerce')

grouped = base_1.groupby(['customer_number'], as_index=False).agg({'CC_income': max,'CC_openDate': max})
final2=pd.merge(final2,grouped,on='customer_number',how='left')

final2["CC_months"]= (pd.to_datetime('today') - final2.CC_openDate)/np.timedelta64(1, 'M')
final2.loc[final2.CC_months>40,'CC_months'] = 40
final2=pd.merge(final2,gp1,on='customer_number',how='left')
del gp1,grouped

gc.collect()

final2.Age=pd.to_numeric(final2["Age"],errors='coerce')
final2['bureau']='Cibil'
s=[]
for i,j in zip(final2["flg"],final2["max_EMI"]):
    if (i==1):
        temp=(j/0.65)
        s.append(temp)
    else:
        s.append(j/0.5)

final2["emi_income"]=s

final2['CC_income_v'] = final2['CC_income']*(1+.08/12)**final2.CC_months
final2['emi_income_v'] = final2['emi_income']*(1+.08/12)**final2.mth
final2['final_income'] = final2[['CC_income_v','emi_income_v']].max(axis=1)

final2['dbr']=pd.to_numeric(final2['m0']/final2['final_income']*100,errors='coerce')

del bur_data
gc.collect()
final2 = final2[['customer_number','final_income','dbr','max_EMI']]
final2 = final2.drop_duplicates('customer_number')
final2.head()

trad = trad.merge(final2, on='customer_number', how = "left")
del final2
gc.collect()

sum_col = ['no_of_trades','live_trad','total_pl','live_pl','total_bl','live_bl','total_cc','live_cc',
       'total_cl','live_cl','total_hl','live_hl','total_al','live_al','new_pl_3m', 'new_pl_6m',
       'new_bl_3m', 'new_bl_6m', 'new_cc_3m', 'new_cc_6m','sum_pl_limit','active_sum_pl_limit',
       'sum_pl_limit_last_6m','sum_bl_limit','active_sum_bl_limit','sum_bl_limit_last_6m','sum_cc_limit',
       'active_sum_cc_limit','sum_cc_limit_last_6m','pl_remaining_balance','bl_remaining_balance',
       'al_remaining_balance', 'hl_remaining_balance','No_of_ALHLLAPPLCCBLSBL','TotalOverDueAmount',
       '1_months_dpd_instances','2_months_dpd_instances','3_months_dpd_instances','6_months_dpd_instances',
       '9_months_dpd_instances','12_months_dpd_instances']

max_col = ['score','Age', "Income" ,'max_EMI','derog','Vintage','pl_vintage','live_pl_vintage',
           'pl_last_active','pl_last_closed','bl_vintage','live_bl_vintage','cc_vintage','live_cc_vintage',
          'cl_vintage','live_cl_vintage','hl_vintage','live_hl_vintage','al_vintage', 'live_al_vintage',
           'active_max_pl_limit','max_pl_limit','max_pl_limit_last_6m','active_max_bl_limit','max_bl_limit',
           'max_bl_limit_last_6m','active_max_cc_limit','max_cc_limit','max_cc_limit_last_6m','max_pl_remaining_balance_per',
           'max_bl_remaining_balance_per','max_al_remaining_balance_per','max_hl_remaining_balance_per',
          'max_credit_utilization','RBL_ultrahni','final_income','dbr','1_months_max_dpd','2_months_max_dpd',
           '3_months_max_dpd','6_months_max_dpd','9_months_max_dpd','12_months_max_dpd','weighted_dpd_6m',
           'weighted_dpd_12m','pl_weighted_dpd_6m','pl_weighted_dpd_12m','bl_weighted_dpd_6m',
           'bl_weighted_dpd_12m','cl_weighted_dpd_6m','cl_weighted_dpd_12m','cc_weighted_dpd_6m',
           'cc_weighted_dpd_12m','max_dpd_ever','30_plus_ever_count']

min_col = ['pl_last_active','pl_last_closed','cc_last_active','cc_last_closed','cl_last_active','cl_last_closed',
          'hl_last_active','hl_last_closed','al_last_active', 'al_last_closed']

mean_col = ['mean_pl_remaining_balance_per','mean_bl_remaining_balance_per','mean_al_remaining_balance_per',
            'mean_hl_remaining_balance_per','mean_credit_utilization']

count_col = ['guarantor','joint','count_of_active_stpl']

sum_col_1 = dict.fromkeys(sum_col, 'sum')
max_col_1 = dict.fromkeys(max_col, 'max')
min_col_1 = dict.fromkeys(min_col, 'min')
mean_col_1 = dict.fromkeys(mean_col, 'mean')
count_col_1 = dict.fromkeys(count_col, 'count')

d = {**sum_col_1, **max_col_1, **min_col_1, **mean_col_1, **count_col_1}

for col in (sum_col+max_col+min_col+mean_col+count_col):
    trad[col] = pd.to_numeric(trad[col], errors = "coerce")

print("Merge")
ddd = trad.groupby('customer_number', as_index=False).agg(d)
del trad
gc.collect()

print(ddd['customer_number'].nunique(),"\n")

trad1 = pd.concat([trad1, ddd], ignore_index = True)
del ddd
gc.collect()

trad1.reset_index(drop = True, inplace = True)
print(trad1.shape[0], trad1['customer_number'].nunique())
trad1.head()


Feat Started
trad feat
amount Based
DPD Based
DBR
Merge
122456 

122456 122456


,customer_number,no_of_trades,live_trad,total_pl,live_pl,total_bl,live_bl,total_cc,live_cc,total_cl,live_cl,total_hl,live_hl,total_al,live_al,new_pl_3m,new_pl_6m,new_bl_3m,new_bl_6m,new_cc_3m,new_cc_6m,sum_pl_limit,active_sum_pl_limit,sum_pl_limit_last_6m,sum_bl_limit,active_sum_bl_limit,sum_bl_limit_last_6m,sum_cc_limit,active_sum_cc_limit,sum_cc_limit_last_6m,pl_remaining_balance,bl_remaining_balance,al_remaining_balance,hl_remaining_balance,No_of_ALHLLAPPLCCBLSBL,TotalOverDueAmount,1_months_dpd_instances,2_months_dpd_instances,3_months_dpd_instances,6_months_dpd_instances,9_months_dpd_instances,12_months_dpd_instances,score,Age,Income,max_EMI,derog,Vintage,pl_vintage,live_pl_vintage,pl_last_active,pl_last_closed,bl_vintage,live_bl_vintage,cc_vintage,live_cc_vintage,cl_vintage,live_cl_vintage,hl_vintage,live_hl_vintage,al_vintage,live_al_vintage,active_max_pl_limit,max_pl_limit,max_pl_limit_last_6m,active_max_bl_limit,max_bl_limit,max_bl_limit_last_6m,active_max_cc_limit,max_cc_limit,max_cc_limit_last_6m,max_pl_remaining_balance_per,max_bl_remaining_balance_per,max_al_remaining_balance_per,max_hl_remaining_balance_per,max_credit_utilization,RBL_ultrahni,final_income,dbr,1_months_max_dpd,2_months_max_dpd,3_months_max_dpd,6_months_max_dpd,9_months_max_dpd,12_months_max_dpd,weighted_dpd_6m,weighted_dpd_12m,pl_weighted_dpd_6m,pl_weighted_dpd_12m,bl_weighted_dpd_6m,bl_weighted_dpd_12m,cl_weighted_dpd_6m,cl_weighted_dpd_12m,cc_weighted_dpd_6m,cc_weighted_dpd_12m,max_dpd_ever,30_plus_ever_count,cc_last_active,cc_last_closed,cl_last_active,cl_last_closed,hl_last_active,hl_last_closed,al_last_active,al_last_closed,mean_pl_remaining_balance_per,mean_bl_remaining_balance_per,mean_al_remaining_balance_per,mean_hl_remaining_balance_per,mean_credit_utilization,guarantor,joint,count_of_active_stpl
0,6000063936,6,2,5,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,8400.0,7000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9663.0,0.0,0.0,0.0,5,9661.0,0,2,4,10,16,21,638.0,28.0,1000.000000,2773.0,0,27.0,19.0,19.0,576.0,418.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,5000.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.07450,0.0,0.00000,0.0,NaN,0,6168.092486,39.850246,0,506,506,506,506,506,1163.67,2223.92,1163.67,2223.92,0.0,0.0,0.0,0.0,0.00,0.00,506,16,NaN,NaN,NaN,556.0,NaN,NaN,NaN,NaN,0.529550,0.0,0.000000,0.0,NaN,0,0,2
1,6000073407,6,0,2,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,541000.0,356000.0,0.0,0.0,0.0,0.0,96143.0,96143.0,0.0,308036.0,0.0,0.0,0.0,4,437382.0,0,2,4,10,16,22,566.0,42.0,NaN,16285.0,1,194.0,75.0,64.0,NaN,1933.0,0.0,0.0,74.0,74.0,0.0,0.0,0.0,0.0,68.0,0.0,356000.0,356000.0,0.0,0.0,0.0,0.0,96143.0,96143.0,0.0,0.86527,0.0,0.00000,0.0,NaN,0,50165.505213,16.511346,0,900,900,900,900,900,2250.00,4950.00,2250.00,4950.00,0.0,0.0,0.0,0.0,528.67,1166.83,900,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.144212,0.0,0.000000,0.0,NaN,0,0,0
2,6000120082,5,3,2,2,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,75675.0,75675.0,0.0,0.0,0.0,0.0,43844.0,43844.0,0.0,71023.0,0.0,0.0,0.0,3,42753.0,0,3,7,18,28,39,615.0,28.0,21931.000000,12278.0,1,24.0,18.0,18.0,552.0,NaN,0.0,0.0,18.0,18.0,20.0,0.0,0.0,0.0,0.0,0.0,72175.0,72175.0,0.0,0.0,0.0,0.0,43844.0,43844.0,0.0,1.00000,0.0,0.00000,0.0,1.0,0,26949.906329,39.068781,0,507,507,507,507,507,1166.00,2229.83,1166.00,2229.83,0.0,0.0,0.0,0.0,26.33,62.75,507,16,543.0,NaN,NaN,365.0,NaN,NaN,NaN,NaN,0.387109,0.0,0.000000,0.0,1.0,0,0,1
3,6000291617,10,5,7,3,0,0,0,0,2,2,0,0,0,0,1,1,0,0,0,0,37500.0,28500.0,25000.0,0.0,0.0,0.0,0.0,0.0,0.0,28380.0,0.0,0.0,0.0,7,14710.0,3,6,9,18,27,31,626.0,28.2,18163.750000,16010.0,1,57.0,51.0,12.0,77.0,383.0,0.0,0.0,0.0,0.0,16.0,16.0,0.0,0.0,0.0,0.0,25000.0,25000.0,25000.0,0.0,0.0,0.0,0.0,0.0,0.0,2.24000,0.0,0.00000,0.0,NaN,0,32020.000000,50.000000,294,636,636,636,636,636,1488.83,2943.08,850.33,1190.58,0.0,0.0,0.0,0.0,0.00,0.00,636,20,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN,0.406268,0.0,0.000000,0.0,NaN,0,0,2
4,6000294824,2,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,815039.0,0.0,1,589423.0,0,1,2,6,12,18,595.0,31.0,54437.4

In [7]:
trad1.to_parquet('bureau_features', compression = 'gzip', index =False)

In [8]:
trad1

,customer_number,no_of_trades,live_trad,total_pl,live_pl,total_bl,live_bl,total_cc,live_cc,total_cl,live_cl,total_hl,live_hl,total_al,live_al,new_pl_3m,new_pl_6m,new_bl_3m,new_bl_6m,new_cc_3m,new_cc_6m,sum_pl_limit,active_sum_pl_limit,sum_pl_limit_last_6m,sum_bl_limit,active_sum_bl_limit,sum_bl_limit_last_6m,sum_cc_limit,active_sum_cc_limit,sum_cc_limit_last_6m,pl_remaining_balance,bl_remaining_balance,al_remaining_balance,hl_remaining_balance,No_of_ALHLLAPPLCCBLSBL,TotalOverDueAmount,1_months_dpd_instances,2_months_dpd_instances,3_months_dpd_instances,6_months_dpd_instances,9_months_dpd_instances,12_months_dpd_instances,score,Age,Income,max_EMI,derog,Vintage,pl_vintage,live_pl_vintage,pl_last_active,pl_last_closed,bl_vintage,live_bl_vintage,cc_vintage,live_cc_vintage,cl_vintage,live_cl_vintage,hl_vintage,live_hl_vintage,al_vintage,live_al_vintage,active_max_pl_limit,max_pl_limit,max_pl_limit_last_6m,active_max_bl_limit,max_bl_limit,max_bl_limit_last_6m,active_max_cc_limit,max_cc_limit,max_cc_limit_last_6m,max_pl_remaining_balance_per,max_bl_remaining_balance_per,max_al_remaining_balance_per,max_hl_remaining_balance_per,max_credit_utilization,RBL_ultrahni,final_income,dbr,1_months_max_dpd,2_months_max_dpd,3_months_max_dpd,6_months_max_dpd,9_months_max_dpd,12_months_max_dpd,weighted_dpd_6m,weighted_dpd_12m,pl_weighted_dpd_6m,pl_weighted_dpd_12m,bl_weighted_dpd_6m,bl_weighted_dpd_12m,cl_weighted_dpd_6m,cl_weighted_dpd_12m,cc_weighted_dpd_6m,cc_weighted_dpd_12m,max_dpd_ever,30_plus_ever_count,cc_last_active,cc_last_closed,cl_last_active,cl_last_closed,hl_last_active,hl_last_closed,al_last_active,al_last_closed,mean_pl_remaining_balance_per,mean_bl_remaining_balance_per,mean_al_remaining_balance_per,mean_hl_remaining_balance_per,mean_credit_utilization,guarantor,joint,count_of_active_stpl
0,6000063936,6,2,5,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,8400.0,7000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9663.0,0.0,0.0,0.0,5,9661.0,0,2,4,10,16,21,638.0,28.0,1.000000e+03,2773.0,0,27.0,19.0,19.0,576.0,418.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,5000.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.074500,0.0,0.000000,0.0,NaN,0,6.168092e+03,39.850246,0,506,506,506,506,506,1163.67,2223.92,1163.67,2223.92,0.0,0.0,0.0,0.0,0.00,0.00,506,16,NaN,NaN,NaN,556.0,NaN,NaN,NaN,NaN,0.529550,0.0,0.000000,0.0,NaN,0,0,2
1,6000073407,6,0,2,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,541000.0,356000.0,0.0,0.0,0.0,0.0,96143.0,96143.0,0.0,308036.0,0.0,0.0,0.0,4,437382.0,0,2,4,10,16,22,566.0,42.0,NaN,16285.0,1,194.0,75.0,64.0,NaN,1933.0,0.0,0.0,74.0,74.0,0.0,0.0,0.0,0.0,68.0,0.0,356000.0,356000.0,0.0,0.0,0.0,0.0,96143.0,96143.0,0.0,0.865270,0.0,0.000000,0.0,NaN,0,5.016551e+04,16.511346,0,900,900,900,900,900,2250.00,4950.00,2250.00,4950.00,0.0,0.0,0.0,0.0,528.67,1166.83,900,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.144212,0.0,0.000000,0.0,NaN,0,0,0
2,6000120082,5,3,2,2,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,75675.0,75675.0,0.0,0.0,0.0,0.0,43844.0,43844.0,0.0,71023.0,0.0,0.0,0.0,3,42753.0,0,3,7,18,28,39,615.0,28.0,2.193100e+04,12278.0,1,24.0,18.0,18.0,552.0,NaN,0.0,0.0,18.0,18.0,20.0,0.0,0.0,0.0,0.0,0.0,72175.0,72175.0,0.0,0.0,0.0,0.0,43844.0,43844.0,0.0,1.000000,0.0,0.000000,0.0,1.0,0,2.694991e+04,39.068781,0,507,507,507,507,507,1166.00,2229.83,1166.00,2229.83,0.0,0.0,0.0,0.0,26.33,62.75,507,16,543.0,NaN,NaN,365.0,NaN,NaN,NaN,NaN,0.387109,0.0,0.000000,0.0,1.0,0,0,1
3,6000291617,10,5,7,3,0,0,0,0,2,2,0,0,0,0,1,1,0,0,0,0,37500.0,28500.0,25000.0,0.0,0.0,0.0,0.0,0.0,0.0,28380.0,0.0,0.0,0.0,7,14710.0,3,6,9,18,27,31,626.0,28.2,1.816375e+04,16010.0,1,57.0,51.0,12.0,77.0,383.0,0.0,0.0,0.0,0.0,16.0,16.0,0.0,0.0,0.0,0.0,25000.0,25000.0,25000.0,0.0,0.0,0.0,0.0,0.0,0.0,2.240000,0.0,0.000000,0.0,NaN,0,3.202000e+04,50.000000,294,636,636,636,636,636,1488.83,2943.08,850.33,1190.58,0.0,0.0,0.0,0.0,0.00,0.00,636,20,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN,0.406268,0.0,0.000000,0.0,NaN,0,0,2
4,6000294824,2,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,815039.0,0.0,1,589423.0,0,1,2,6,12,18,595.0,31

In [7]:
base = pd.read_parquet('cibil_base_aug_jul.parquet.gzip')

In [9]:
base.applicationCreditReportId = np.where(base.applicationCreditReportId.isna(),base.panId,base.applicationCreditReportId)

In [10]:
base = base[['mobilePhone','applicationCreditReportId']].drop_duplicates(subset = 'applicationCreditReportId')

In [15]:
enq = pd.merge(enq,base, on = 'applicationCreditReportId', how = 'left')

In [16]:
enq

,applicationCreditReportId,enquiryDate,enquiryType,amount,bureauDate,mobilePhone
0,fd56f252-7d13-5c85-b36f-2f514763738a,2022-08-04,Personal Loan,95000,2022-08-04,8299458755
1,fd56f252-7d13-5c85-b36f-2f514763738a,2023-01-03,Consumer Loan,4000,2023-01-03,8299458755
2,3a942953-803d-5c6a-be32-6731765432e6,2023-02-18,Personal Loan,3000,2023-02-18,8438996213
3,3a942953-803d-5c6a-be32-6731765432e6,2022-04-13,Personal Loan,5000,2022-04-13,8438996213
4,3a942953-803d-5c6a-be32-6731765432e6,2022-05-05,Two-Wheeler Loan,86000,2022-05-05,8438996213
...,...,...,...,...,...,...
2046004,8264826452_19May2023,2014-10-09 00:00:00,Gold Loan,50000,2014-10-09,8264826452
2046005,8264826452_19May2023,2014-05-17 00:00:00,Two-Wheeler Loan,47000,2014-05-17,8264826452
2046006,8264826452_19May2023,2012-11-07 00:00:00,Consumer Loan,25000,2012-11-07,8264826452
2046007,8264826452_19May2023,2011-06-15 00:00:00,Two-Wheeler Loan,43000,2011-06-15,8264826452


In [17]:
enq.rename(columns = {'enquiryDate':'enquirydate','enquiryType':'enquirytype','bureauDate':'Bureau_Date'}, inplace = True)




enq['customer_number'] = enq['mobilePhone']
del enq['mobilePhone']
gc.collect()


0

In [18]:
enq1 = pd.DataFrame()
enq['Bureau_Date'] = pd.to_datetime(enq['Bureau_Date'])
enq['enquirydate'] = pd.to_datetime(enq['enquirydate'])
enq['amount'] = pd.to_numeric(enq['amount'], errors ="coerce")
enq['days_enquired'] = (enq['Bureau_Date'] - enq['enquirydate']).dt.days
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=30),'PL_Enq_1M'] = 1
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=30),'BL_Enq_1M'] = 1
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=60),'PL_Enq_2M'] = 1
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=60),'BL_Enq_2M'] = 1
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=90),'PL_Enq_3M'] = 1
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=90),'BL_Enq_3M'] = 1
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=180),'PL_Enq_6M'] = 1
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=180),'BL_Enq_6M'] = 1
enq.loc[(enq['enquirytype'].isin(cc))&(enq['days_enquired']<=30),'CC_Enq_1M'] = 1
enq.loc[(enq['enquirytype'].isin(cc))&(enq['days_enquired']<=60),'CC_Enq_2M'] = 1
enq.loc[(enq['enquirytype'].isin(cc))&(enq['days_enquired']<=90),'CC_Enq_3M'] = 1
enq.loc[(enq['enquirytype'].isin(cc))&(enq['days_enquired']<=180),'CC_Enq_6M'] = 1
# enq.loc[(enq['enquirytype'].isin(al))&(enq['days_enquired']<=30),'AL_Enq_1M'] = 1
# enq.loc[(enq['enquirytype'].isin(al))&(enq['days_enquired']<=60),'AL_Enq_2M'] = 1
# enq.loc[(enq['enquirytype'].isin(al))&(enq['days_enquired']<=90),'AL_Enq_3M'] = 1
# enq.loc[(enq['enquirytype'].isin(al))&(enq['days_enquired']<=180),'AL_Enq_6M'] = 1
# enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=30),'HL_Enq_1M'] = 1
# enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=60),'HL_Enq_2M'] = 1
# enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=90),'HL_Enq_3M'] = 1
# enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=180),'HL_Enq_6M'] = 1
enq.loc[(enq['enquirytype'].isin(cl))&(enq['days_enquired']<=30),'CL_Enq_1M'] = 1
enq.loc[(enq['enquirytype'].isin(cl))&(enq['days_enquired']<=60),'CL_Enq_2M'] = 1
enq.loc[(enq['enquirytype'].isin(cl))&(enq['days_enquired']<=90),'CL_Enq_3M'] = 1
enq.loc[(enq['enquirytype'].isin(cl))&(enq['days_enquired']<=180),'CL_Enq_6M'] = 1
# enq.loc[(enq['enquirytype'].isin(gl))&(enq['days_enquired']<=30),'GL_Enq_1M'] = 1
# enq.loc[(enq['enquirytype'].isin(gl))&(enq['days_enquired']<=60),'GL_Enq_2M'] = 1
# enq.loc[(enq['enquirytype'].isin(gl))&(enq['days_enquired']<=90),'GL_Enq_3M'] = 1
# enq.loc[(enq['enquirytype'].isin(gl))&(enq['days_enquired']<=180),'GL_Enq_6M'] = 1
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=30),'UN_Enq_1M'] = 1
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=60),'UN_Enq_2M'] = 1
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=90),'UN_Enq_3M'] = 1
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=180),'UN_Enq_6M'] = 1
enq.loc[(enq['days_enquired']<=30),'All_Enq_1M'] = 1
enq.loc[(enq['days_enquired']<=60),'All_Enq_2M'] = 1
enq.loc[(enq['days_enquired']<=90),'All_Enq_3M'] = 1
enq.loc[(enq['days_enquired']<=180),'All_Enq_6M'] = 1
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=30),'PL_Enq_Amount_1M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=60),'PL_Enq_Amount_2M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=90),'PL_Enq_Amount_3M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=180),'PL_Enq_Amount_6M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=30),'Bl_Enq_Amount_1M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=60),'Bl_Enq_Amount_2M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=90),'Bl_Enq_Amount_3M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=180),'Bl_Enq_Amount_6M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=30),'Hl_Enq_Amount_1M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=60),'Hl_Enq_Amount_2M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=90),'Hl_Enq_Amount_3M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=180),'Hl_Enq_Amount_6M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=30),'UN_Enq_Amount_1M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=60),'UN_Enq_Amount_2M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=90),'UN_Enq_Amount_3M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=180),'UN_Enq_Amount_6M_sum'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=30),'PL_Enq_Amount_1M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=60),'PL_Enq_Amount_2M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=90),'PL_Enq_Amount_3M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(pl))&(enq['days_enquired']<=180),'PL_Enq_Amount_6M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=30),'Bl_Enq_Amount_1M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=60),'Bl_Enq_Amount_2M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=90),'Bl_Enq_Amount_3M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(bl))&(enq['days_enquired']<=180),'Bl_Enq_Amount_6M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=30),'Hl_Enq_Amount_1M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=60),'Hl_Enq_Amount_2M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=90),'Hl_Enq_Amount_3M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(hl))&(enq['days_enquired']<=180),'Hl_Enq_Amount_6M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=30),'UN_Enq_Amount_1M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=60),'UN_Enq_Amount_2M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=90),'UN_Enq_Amount_3M_max'] = enq['amount']
enq.loc[(enq['enquirytype'].isin(unsecured))&(enq['days_enquired']<=180),'UN_Enq_Amount_6M_max'] = enq['amount']
sum_col = ['PL_Enq_Amount_1M_sum','PL_Enq_Amount_2M_sum','PL_Enq_Amount_3M_sum','PL_Enq_Amount_6M_sum',
       'Bl_Enq_Amount_1M_sum','Bl_Enq_Amount_2M_sum','Bl_Enq_Amount_3M_sum','Bl_Enq_Amount_6M_sum',
       'Hl_Enq_Amount_1M_sum','Hl_Enq_Amount_2M_sum','Hl_Enq_Amount_3M_sum','Hl_Enq_Amount_6M_sum',
       'UN_Enq_Amount_1M_sum','UN_Enq_Amount_2M_sum','UN_Enq_Amount_3M_sum','UN_Enq_Amount_6M_sum',
       'PL_Enq_1M', 'BL_Enq_1M', 'PL_Enq_2M', 'BL_Enq_2M','PL_Enq_3M', 'BL_Enq_3M', 'PL_Enq_6M',
       'BL_Enq_6M', 'All_Enq_1M','All_Enq_2M', 'All_Enq_3M', 'All_Enq_6M', 'CC_Enq_1M', 'CC_Enq_2M',
       'CC_Enq_3M', 'CC_Enq_6M', 'CL_Enq_1M', 'CL_Enq_2M', 'CL_Enq_3M','CL_Enq_6M', 'UN_Enq_1M',
       'UN_Enq_2M', 'UN_Enq_3M', 'UN_Enq_6M']
max_col = ['PL_Enq_Amount_1M_max','PL_Enq_Amount_2M_max','PL_Enq_Amount_3M_max','PL_Enq_Amount_6M_max',
           'Bl_Enq_Amount_1M_max','Bl_Enq_Amount_2M_max','Bl_Enq_Amount_3M_max','Bl_Enq_Amount_6M_max',
           'Hl_Enq_Amount_1M_max','Hl_Enq_Amount_2M_max','Hl_Enq_Amount_3M_max','Hl_Enq_Amount_6M_max',
           'UN_Enq_Amount_1M_max','UN_Enq_Amount_2M_max','UN_Enq_Amount_3M_max','UN_Enq_Amount_6M_max']
sum_col = dict.fromkeys(sum_col, "sum")
max_col = dict.fromkeys(max_col, "max")
d = {**sum_col, **max_col}
del sum_col, max_col
data_list = enq.groupby('customer_number', as_index=False).agg(d)
del enq
gc.collect()
print(data_list['customer_number'].nunique(),'\n')
enq1 = pd.concat([enq1,data_list],ignore_index=True)
del data_list
gc.collect()
enq1.reset_index(drop = True, inplace = True)
print(enq1.shape[0],enq1['customer_number'].nunique())
enq1.head()


157340 

157340 157340


,customer_number,PL_Enq_Amount_1M_sum,PL_Enq_Amount_2M_sum,PL_Enq_Amount_3M_sum,PL_Enq_Amount_6M_sum,Bl_Enq_Amount_1M_sum,Bl_Enq_Amount_2M_sum,Bl_Enq_Amount_3M_sum,Bl_Enq_Amount_6M_sum,Hl_Enq_Amount_1M_sum,Hl_Enq_Amount_2M_sum,Hl_Enq_Amount_3M_sum,Hl_Enq_Amount_6M_sum,UN_Enq_Amount_1M_sum,UN_Enq_Amount_2M_sum,UN_Enq_Amount_3M_sum,UN_Enq_Amount_6M_sum,PL_Enq_1M,BL_Enq_1M,PL_Enq_2M,BL_Enq_2M,PL_Enq_3M,BL_Enq_3M,PL_Enq_6M,BL_Enq_6M,All_Enq_1M,All_Enq_2M,All_Enq_3M,All_Enq_6M,CC_Enq_1M,CC_Enq_2M,CC_Enq_3M,CC_Enq_6M,CL_Enq_1M,CL_Enq_2M,CL_Enq_3M,CL_Enq_6M,UN_Enq_1M,UN_Enq_2M,UN_Enq_3M,UN_Enq_6M,PL_Enq_Amount_1M_max,PL_Enq_Amount_2M_max,PL_Enq_Amount_3M_max,PL_Enq_Amount_6M_max,Bl_Enq_Amount_1M_max,Bl_Enq_Amount_2M_max,Bl_Enq_Amount_3M_max,Bl_Enq_Amount_6M_max,Hl_Enq_Amount_1M_max,Hl_Enq_Amount_2M_max,Hl_Enq_Amount_3M_max,Hl_Enq_Amount_6M_max,UN_Enq_Amount_1M_max,UN_Enq_Amount_2M_max,UN_Enq_Amount_3M_max,UN_Enq_Amount_6M_max
0,6000012026,30000.0,30000.0,30000.0,30000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,530000.0,530000.0,530000.0,530000.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,7.0,7.0,7.0,7.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,30000.0,30000.0,30000.0,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250000.0,250000.0,250000.0,250000.0
1,6000063936,68000.0,68000.0,68000.0,68000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,148503.0,148503.0,148503.0,148503.0,6.0,0.0,6.0,0.0,6.0,0.0,6.0,0.0,13.0,13.0,13.0,13.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,13.0,13.0,13.0,13.0,30000.0,30000.0,30000.0,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,50000.0,50000.0,50000.0
2,6000108004,175500.0,175500.0,175500.0,175500.0,100000.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0,1078492.0,1078492.0,1078492.0,1078492.0,5.0,1.0,5.0,1.0,5.0,1.0,5.0,1.0,22.0,22.0,22.0,22.0,1.0,1.0,1.0,1.0,12.0,12.0,12.0,12.0,22.0,22.0,22.0,22.0,150000.0,150000.0,150000.0,150000.0,100000.0,100000.0,100000.0,100000.0,NaN,NaN,NaN,NaN,250000.0,250000.0,250000.0,250000.0
3,6000131474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25999.0,25999.0,25999.0,25999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13999.0,13999.0,13999.0,13999.0
4,6000161399,25000.0,25000.0,25000.0,25000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75000.0,75000.0,75000.0,75000.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,25000.0,25000.0,25000.0,25000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25000.0,25000.0,25000.0,25000.0


In [19]:
enq1.to_parquet('enq_features', compression = 'gzip', index =False)